# Getting Started With OpenAI Structured Outputs

## Introduction

In August, 2024, OpenAI announced a powerful new feature, Structured Outputs, in their API. With Structured Outputs, as the name suggests, you can ensure LLMs will generate responses only in a format you specify. This capability will make it significantly easier to build applications that require precise data formatting. In this tutorial, you will learn how to get started with this new feature, understand its new syntax and explore its key applications.

## Importance of Structured Outputs in AI Applications

Deterministic responses, or in other words, responses in the same format is crucial for a lot of tasks such as data entry, information retrieval, question answering, multi-step workflows and so on. I am sure you must know how LLMs can generate outputs in wildly different formats even if the prompt is the same. 

For example, consider this simple hotel reviews sentiment classifier:

In [6]:
from openai import OpenAI

client = OpenAI()
MODEL = "gpt-4o"

SYSTEM_PROMPT = "You are a sentiment classifier assistant."
PROMPT_TEMPLATE = """
    Classify the sentiment of the following hotel review as positive, negative, or neutral:\n\n{review}
"""

# Function to classify sentiment using OpenAI's chat completions API
def classify_sentiment(review):
    response = client.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": PROMPT_TEMPLATE.format(review=review)}
        ],
    )
    return response.choices[0].message.content.strip()

# List of hotel reviews
reviews = [
    "The room was clean and the staff was friendly.",
    "The location was terrible and the service was slow.",
    "The food was amazing but the room was too small.",
]

# Classify sentiment for each review and print the results
for review in reviews:
    sentiment = classify_sentiment(review)
    print(f"Review: {review}\nSentiment: {sentiment}\n")

Review: The room was clean and the staff was friendly.
Sentiment: Positive

Review: The location was terrible and the service was slow.
Sentiment: Negative

Review: The food was amazing but the room was too small.
Sentiment: The sentiment of the review is neutral.



Even though the first two responses were in the same single-word format, the last one is an entire sentence. If some other downstream application depended on the output of the above code, it would have crashed as it would have been expecting a single-word response.

We can fix this problem with some prompt engineering but it is a time-consuming, iterative process. Even with a perfect prompt, we can't be 100% sure the responses will conform to our format in future requests. Unless, of course, we use Structured Outputs:

```python
def classify_sentiment_with_structured_outputs(review):
    ...

# Classify sentiment for each review with Structured Outputs
for review in reviews:
    sentiment = classify_sentiment_with_structured_outputs(review)
    print(f"Review: {review}\nSentiment: {sentiment}\n")
```


```python
Review: The room was clean and the staff was friendly.
Sentiment: {"sentiment":"positive"}

Review: The location was terrible and the service was slow.
Sentiment: {"sentiment":"negative"}

Review: The food was amazing but the room was too small.
Sentiment: {"sentiment":"neutral"}
```

With a new function, `classify_sentiment_with_structured_outputs`, the responses are all in the same format. 

## Getting Started  With Structured Outputs

In this section, we will explain how Structured Outputs work by continuing th example of the review sentiments analyzer. 

### Setting Up Your Environment

#### Prerequisites

Before you begin, ensure you have the following:

- Python 3.7 or later installed on your system.
- An OpenAI API key. You can obtain this by signing up on the [OpenAI website](https://openai.com/).

#### Setting Up the OpenAI API

1. **Install the OpenAI Python package**:
   Open your terminal and run the following command to install or update the OpenAI Python package to the latest version:
   ```bash
   pip install -U openai
   ```

2. **Set up your API key**:
   You can set your API key as an environment variable or directly in your code. To set it as an environment variable, run:
   ```bash
   export OPENAI_API_KEY='your-api-key'
   ```

3. **Verify the installation**:
   Create a simple Python script to verify the installation:

In [16]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say hello!"}
    ],
    max_tokens=5
)

print(response.choices[0].message.content.strip())

Hello! How can I


Run the script to ensure everything is set up correctly. You should see the model's response printed in the terminal.

In addition to the OpenAI package, you will need the Pydantic library to define and validate JSON schemas for Structured Outputs. Install it using pip:

```bash
pip install pydantic
```

With these steps, your environment is now set up to use OpenAI's Structured Outputs feature. 

### Defining an output schema using Pydantic

To use Structured Outputs, you need to define the expected output structure using Pydantic models. Pydantic is a data validation and settings management library for Python, which allows you to define data models using Python type annotations. These models can then be used to enforce the structure of the outputs generated by OpenAI's models.

Here is an example Pydantic model for specifying the format for our review sentiment classifier:

In [19]:
from pydantic import BaseModel
from typing import Literal

class SentimentResponse(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"]

In this example:
- `SentimentResponse` is a Pydantic model that defines the expected structure of the output.
- The model has a single field `sentiment`, which can only take one of three literal values: "positive", "negative", or "neutral".

When we pass this model as part of our OpenAI API requests, the outputs will be only one of the words we provided. 

Let's see how.

### Using the `parse` helper

To enforce our Pydantic schema in OpenAI requests, all we have to do is pass it to the `response_format` parameter of the chat completions API. Roughly, here is what it looks like:

```python
response = client.beta.chat.completions.parse(
    model=MODEL,
    messages=[...],
    response_format=SentimentResponse
)
```

If you notice, instead of using `client.chat.completions.create`, we are using `client.beta.chat.completions.parse` method. `.parse()` is a new method in the Chat Completions API specifically written for Structured Outputs. 

Now, let's put everything together by rewriting the reviews sentiment classifier with Structured Outputs. First, we make the necessary imports, define the Pydantic model, the system prompt and a prompt template:

In [27]:
from openai import OpenAI
from pydantic import BaseModel
from typing import Literal


class SentimentResponse(BaseModel):
    sentiment: Literal["positive", "negative", "neutral"]


client = OpenAI()
MODEL = "gpt-4o-mini"

SYSTEM_PROMPT = "You are a sentiment classifier assistant."
PROMPT_TEMPLATE = """
    Classify the sentiment of the following hotel review as positive, negative, or neutral:\n\n{review}
"""

Then, we write a new function that uses the `.parse` helper method:

In [28]:
# Function to classify sentiment using OpenAI's chat completions API with structured outputs
def classify_sentiment_with_structured_outputs(review):
    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": PROMPT_TEMPLATE.format(review=review)},
        ],
        response_format=SentimentResponse
    )
    return response.choices[0].message

The important line in the function is `response_format=SentimentResponse`, which is what actually enables Structured Outputs. 

Let's test it on one of the reviews:

In [29]:

# List of hotel reviews
reviews = [
    "The room was clean and the staff was friendly.",
    "The location was terrible and the service was slow.",
    "The food was amazing but the room was too small.",
]

result = classify_sentiment_with_structured_outputs(reviews[0])
print(result.content)

{"sentiment":"positive"}


Here, `result` is a message object:

In [ ]:
type(result)

openai.types.chat.parsed_chat_completion.ParsedChatCompletionMessage[SentimentResponse]

Apart from its `,content` attribute which retrieves the response, it has a `.parsed` attribute that returns the parsed information as a class:

In [34]:
result.parsed

SentimentResponse(sentiment='positive')

As you can see, we have got an instance of the `SentimentResponse` class. This means, we can access the sentiment as a string instead of a dictionary using the `.sentiment` attribute:

In [35]:
result.parsed.sentiment

'positive'

## Nesting Pydantic Models For Defining Complex Schemas

In some cases, you may need to define more complex output structures that involve nested data. Pydantic allows you to nest models within each other, enabling you to create intricate schemas that can handle a variety of use cases. This is particularly useful when dealing with hierarchical data or when you need to enforce a specific structure for complex outputs.

Let's consider an example where we need to extract detailed user information, including their name, contact details, and a list of addresses. Each address should include fields for the street, city, state, and zip code. This requires multiple Pydantic models to build a complex response schema.

__Step 1: Define the Pydantic Models__

First, we define the Pydantic models for the address and user information:


In [45]:
from pydantic import BaseModel
from typing import List

# Define the Pydantic model for an address
class Address(BaseModel): 
    street: str 
    city: str 
    state: str 
    zip_code: str

# Define the Pydantic model for user information
class UserInfo(BaseModel): 
    name: str 
    email: str 
    phone: str 
    addresses: List[Address]

In this example:
    
- `Address` is a Pydantic model that defines the structure of an address.
- `UserInfo` is a Pydantic model that includes a list of `Address` objects, along with fields for the user's name, email, and phone number.

__Step 2: Use the Nested Pydantic Models in API Calls__

Next, we use these nested Pydantic models to enforce the output structure in an OpenAI API call:

In [51]:
SYSTEM_PROMPT = "You are a user information extraction assistant."
PROMPT_TEMPLATE = """ Extract the user information from the following text:\n\n{text}"""


# Function to extract user information using OpenAI's chat completions API with structured outputs
def extract_user_info(text):
    response = client.beta.chat.completions.parse(
        model=MODEL,
        messages=[
            {"role": "system", "content": SYSTEM_PROMPT},
            {"role": "user", "content": PROMPT_TEMPLATE.format(text=text)},
        ],
        response_format=UserInfo
    )

    return response.choices[0].message


# Example text containing user information

text = """John DoeEmail: john.doe@example.comPhone: 123-456-7890Addresses:- 123 Main St, Springfield, IL, 62701- 456 Elm St, Shelbyville, IL, 62702"""

# Extract user information and print the results
user_info = extract_user_info(text)

The syntax is the same as before, even though we are using a complex Pydantic format. Let's look at the format:

In [62]:
import json

data = json.loads(user_info.content)
pretty_response = json.dumps(data, indent=2)
print(pretty_response)

{
  "name": "John Doe",
  "email": "john.doe@example.com",
  "phone": "123-456-7890",
  "addresses": [
    {
      "street": "123 Main St",
      "city": "Springfield",
      "state": "IL",
      "zip_code": "62701"
    },
    {
      "street": "456 Elm St",
      "city": "Shelbyville",
      "state": "IL",
      "zip_code": "62702"
    }
  ]
}


As you can see, the model correctly capture a single user's information along with their two separate addresses. 

In short, by nesting Pydantic models, you can define complex schemas that handle hierarchical data and enforce specific structures for intricate outputs.

## Function Calling with Structured Outputs

## Best Practices

### Handling Refusals

When using Structured Outputs, the model may refuse requests for safety reasons. Since refusal messages don't conform to any custom schema, the OpenAI API has a new field `refusal` to indicate a model's refusal. 

This is useful for properly handling errors and render refusals directly in UIs:

## Conclusion

### Table of Contents

1. **Introduction**
   - Overview of Structured Outputs
   - Importance of Structured Outputs in AI Applications

2. **Environment Setup**
   - Prerequisites
   - Setting Up the OpenAI API
   - Installing Required Libraries (Python, Node.js)

3. **Understanding Structured Outputs**
   - What are Structured Outputs?
   - Key Features and Benefits

4. **Getting Started with Structured Outputs**
   - Enabling Structured Outputs in API Calls
   - Using JSON Schemas
   - Function Calling with Structured Outputs

5. **Common Use Cases**
   - Extracting Structured Data from Unstructured Inputs
   - Generating User Interfaces Based on User Intent
   - Populating Databases with Extracted Content

6. **Best Practices**
   - Ensuring Schema Compliance
   - Handling Refusals and Errors

7. **Conclusion**
   - Recap of Key Points
   - Additional Resources and Documentation